# Basic liquid handling

In this notebook, you will learn how to use PyLabRobot to move water from one range of wells to another.

**Note: before running this notebook, you should have**:

- Installed PyLabRobot and the USB driver as described in [the installation guide](installation).
- Connected the Hamilton to your computer using the USB cable.

Video of what this code does:

<iframe width="640" height="360" src="https://www.youtube.com/embed/NN6ltrRj3bU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

## Setting up a connection with the robot

Start by importing the {class}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler` class, which will serve as a front end for all liquid handling operations.

Backends serve as communicators between `LiquidHandler`s and the actual hardware. Since we are using a Hamilton STAR, we also import the {class}`~pylabrobot.liquid_handling.backends.STAR` backend.

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
!lsusb

Bus 000 Device 001: ID 0409:005a NEC Corporation Hub 
Bus 000 Device 002: ID 18a5:0302 18a5 STORE N GO  Serial: 21062855540071
Bus 000 Device 000: ID 0409:005a NEC Corporation USB 3.1 Bus 
Bus 000 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub 


In [35]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends import Vantage

In addition, import the {class}`~pylabrobot.resources.hamilton.STARLetDeck`, which represents the deck of the Hamilton STAR.

In [36]:
from pylabrobot.resources.hamilton import VantageDeck

In [37]:
backend = Vantage()
lh = LiquidHandler(backend=backend, deck=VantageDeck(size = 1.3))

The final step is to open communication with the robot. This is done using the {func}`~pylabrobot.liquid_handling.LiquidHandler.setup` method.

In [38]:
await lh.setup()

## Creating the deck layout

Now that we have a `LiquidHandler` instance, we can define the deck layout.

The layout in this tutorial will contain five sets of standard volume tips with filter, 1 set of 96 1mL wells, and tip and plate carriers on which these resources are positioned.

Start by importing the relevant objects and variables from the PyLabRobot package. This notebook uses the following resources:

- {class}`~pylabrobot.resources.ml_star.tip_carriers.TIP_CAR_480_A00` tip carrier
- {class}`~pylabrobot.resources.ml_star.plate_carriers.PLT_CAR_L5AC_A00` plate carrier
- {class}`~pylabrobot.resources.corning_costar.plates.Cos_96_DW_1mL` wells
- {class}`~pylabrobot.resources.ml_star.tip_racks.HTF_L` tips

In [39]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    HT_L
)

Then create a tip carrier named `tip carrier`, which will contain tip rack at all 5 positions. These positions can be accessed using `tip_car[x]`, and are 0 indexed.

In [41]:
await lh.backend.core96_initialize(
        x_position=7347-100, # TODO: get trash location from deck.
        y_position=2684, # TODO: get trash location from deck.
        minimal_traverse_height_at_begin_of_command=int(245 * 10),
        minimal_height_at_command_end=int(245 * 10),
        end_z_deposit_position=2420,
      )

'A1HMDIid14er0'

In [42]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = tiprack = HT_L(name='tips_01')

In [26]:
tip_car.unassign()

Use {func}`~pylabrobot.resources.abstract.assign_child_resources` to assign the tip carrier to the deck of the liquid handler. All resources contained by this carrier will be assigned automatically.

In the `rails` parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.

In [43]:
lh.deck.assign_child_resource(tip_car, rails=17)

Repeat this for the plates.

In [44]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='plate_01')


In [45]:
plt_car[1] = Cos_96_DW_1mL(name='plate_02')

In [46]:
lh.deck.assign_child_resource(plt_car, rails=11)

Let's look at a summary of the deck layout using {func}`~pylabrobot.liquid_handling.LiquidHandler.summary`.

In [15]:
lh.summary()

Rail     Resource                   Type                Coordinates (mm)
(8)   ├── plate carrier              PlateCarrier        (257.500, 063.000, 100.000)
      │   ├── plate_01               Plate               (275.500, 146.000, 187.150)
      │   ├── plate_02               Plate               (275.500, 242.000, 187.150)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(14)  ├── tip carrier                TipCarrier          (392.500, 063.000, 100.000)
      │   ├── tips_01                TipRack             (410.400, 145.800, 131.450)
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │   ├── <empty>
      │
(31)  ├── trash                      Trash               (777.500, 185.600, 137.100)



## Picking up tips

Picking up tips is as easy as querying the tips from the tiprack.

In [ ]:
tiprack = lh.deck.get_resource("tips_01")
await lh.pick_up_tips(tiprack["A1:C1"])

somthing happened
tip are there in tip spots


## Aspirating and dispensing

Aspirating and dispensing work similarly to picking up tips: where you use booleans to specify which tips to pick up, with aspiration and dispensing you use floats to specify the volume to aspirate or dispense in $\mu L$.

The cells below move liquid from wells `'A1:C1'` to `'D1:F1'` using channels 1, 2, and 3 using the {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate` and {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense` methods.

In [47]:
await lh.backend.set_loading_cover(cover_open=True)

'I1AMLPid15er0'

In [ ]:
plt_car.unassign()

In [ ]:
plate = lh.deck.get_resource("plate_01")
await lh.aspirate(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

In [ ]:
source_wells = lh.deck.get_resource("plate_01")["A1:C1"]

await lh.aspirate(source_wells)

Coordinate(x=429.75, y=133.75, z=187.15)

After the liquid has been aspirated, dispense it in the wells below. Note that while we specify different wells, we are still using the same channels. This is needed because only these channels contain liquid, of course.

In [ ]:
await lh.dispense(plate["A10:C10"], vols=[100.0, 50.0, 200.0])

In [ ]:
await lh.return_tips()

RuntimeError: No tips have been picked up.

In [50]:
await lh.pick_up_tips96(tiprack)

In [51]:
await lh.drop_tips96(trash)

NameError: name 'trash' is not defined

In [ ]:
lh.deck.get_resource("plate_02")

Plate(name=plate_02, size_x=127.0, size_y=86.0, size_z=42.0, location=(000.000, 000.000, 000.000))

In [ ]:
await lh.aspirate96(lh.deck.get_resource("plate_02"), volume=100)

In [ ]:
await lh.dispense96(lh.deck.get_resource("plate_01"), volume=100)

In [ ]:
await lh.return_tips96()

NoTipError: Channel 0 does not have a tip.

Let's move the liquid back to the original wells.

In [ ]:
await lh.aspirate(plate["D1:F1"], vols=[100.0, 50.0, 200.0])
await lh.dispense(plate["A1:C1"], vols=[100.0, 50.0, 200.0])

INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0ASid0008at0&tm1 1 1 0&xp04330 04330 04330 00000&yp1190 1100 1010 0000&th2450te2450lp1931 1931 1931&ch000 000 000&zl1881 1881 1881&po0100 0100 0100&zu0032 0032 0032&zr06180 06180 06180&zx1831 1831 1831&ip0000 0000 0000&it0 0 0&fp0000 0000 0000&av01072 00551 02110&as1000 1000 1000&ta000 000 000&ba0000 0000 0000&oa000 000 000&lm0 0 0&ll1 1 1&lv1 1 1&zo000 000 000&ld00 00 00&de0020 0020 0020&wt10 10 10&mv00000 00000 00000&mc00 00 00&mp000 000 000&ms1000 1000 1000&mh0000 0000 0000&gi000 000 000&gj0gk0lk0 0 0&ik0000 0000 0000&sd0500 0500 0500&se0500 0500 0500&sz0300 0300 0300&io0000 0000 0000&il00000 00000 00000&in0000 0000 0000&
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Received response: C0ASid0008er00/00
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0DSid0009dm2 2 2&tm1 1 1 0&xp04330 04330 04330 00000&yp1460 1370 1280 0000&zx1871 1871 1871&lp2321 2321 2321&zl1881 1881 1881&po0100 01

## Dropping tips

Finally, you can drop tips anywhere on the deck by using the {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips` method.

In [ ]:
await lh.drop_tips(tiprack["A1:C1"])

INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Sent command: C0TRid0010xp01629 01629 01629 00000&yp1458 1368 1278 0000&tm1 1 1 0&tt01tp1314tz1414th2450ti0
INFO:pylabrobot.liquid_handling.backends.hamilton.STAR:Received response: C0TRid0010er00/00kz381 356 365 000 000 000 000 000vz303 360 368 000 000 000 000 000


In [ ]:
await lh.stop()